In [2]:
import os
import zipfile
import pandas as pd
import re
import json
from pathlib import Path
from datetime import datetime
import emoji

In [9]:
# --- Step 1: Define paths ---
BASE_DIR = Path("../data1")
ZIP_PATH = BASE_DIR / "photos.zip"
IMAGES_DIR = BASE_DIR / "photos"
DATA_FILE = BASE_DIR / "telegram_data.csv"

# --- Step 2: Unzip images ---
if not IMAGES_DIR.exists():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(IMAGES_DIR)
    print("Images extracted.")
else:
    print("Images already extracted.")

# --- Step 3: Load Telegram data ---
try:
    df = pd.read_csv(DATA_FILE)
except:
    df = pd.read_json(DATA_FILE, lines=True)

df = df.rename(columns=str.lower)
print("Loaded messages:", df.shape)
print(df.head())

# --- Step 4: Normalize Amharic text ---
def clean_amharic(text):
    if not isinstance(text, str):
        return ""
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

#df["clean_text"] = df["text"].apply(clean_amharic)
df["clean_text"] = df["message"].apply(clean_amharic)

df["has_image"] = df["media path"].apply(lambda x: os.path.exists(IMAGES_DIR / str(x)) if pd.notna(x) else False)

# --- Step 5: Export cleaned data ---
df_filtered = df[["id", "clean_text", "media path", "has_image"]]
df_filtered.to_csv("../data1/preprocessed_telegram_data.csv", index=False)
print("Saved to /data1/preprocessed_telegram_data.csv")

# --- Step 6: Optional stats ---
price_mentions = df["clean_text"].str.contains(r"(ብር|\d+\s*birr|\d+\s*ብር)", na=False)
location_mentions = df["clean_text"].str.contains(r"(አዲስ\s?አበባ|ቦሌ|መገናኛ)", na=False)

print("Messages with price mentions:", price_mentions.sum())
print("Messages with location mentions:", location_mentions.sum())


Images already extracted.
Loaded messages: (3844, 6)
      channel title channel username     id  \
0  AwasMart-አዋስማርት🎁        @AwasMart  11864   
1  AwasMart-አዋስማርት🎁        @AwasMart  11863   
2  AwasMart-አዋስማርት🎁        @AwasMart  11862   
3  AwasMart-አዋስማርት🎁        @AwasMart  11861   
4  AwasMart-አዋስማርት🎁        @AwasMart  11860   

                                             message                 date  \
0  🤎🚽🚽🚽🚽🚽🚽🚽🚽🚽🚽🚽\n❇️Baby toilet seat/ potty seat\n...  2025-06-22 11:55:19   
1  🔖🔖🔖🔖🔖🔖🔖🔖🔖🔖🔖🔖\nውድ ደንበኞቻችን ሱቃችን ነገ እሁድ ከ4:30-11:...  2025-06-21 16:43:54   
2                  🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪\n❇️Baby suction bowl  2025-06-21 16:43:12   
3                                                NaN  2025-06-21 16:43:12   
4  🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪🍪\n❇️Baby suction bowl\n\n👉 ባለ ሁለት ...  2025-06-21 16:42:32   

                            media path  
0  ../data1/photos\@AwasMart_11864.jpg  
1  ../data1/photos\@AwasMart_11863.jpg  
2                                  NaN  
3  ../data1/photos\@AwasMart_11861.jp

C:\Users\user\AppData\Local\Temp\ipykernel_11068\2457185132.py:45: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  price_mentions = df["clean_text"].str.contains(r"(ብር|\d+\s*birr|\d+\s*ብር)", na=False)
C:\Users\user\AppData\Local\Temp\ipykernel_11068\2457185132.py:46: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  location_mentions = df["clean_text"].str.contains(r"(አዲስ\s?አበባ|ቦሌ|መገናኛ)", na=False)


In [6]:
print(df.columns)


Index(['channel title', 'channel username', 'id', 'message', 'date',
       'media path'],
      dtype='object')
